In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:
data = pd.read_csv('/Users/soukainagadir/Downloads/EEG.machinelearing_data_BRMH.csv')

data.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [6]:
print("Data Shape:", data.shape)

Data Shape: (945, 1149)


In [7]:
print("Data info:", data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Columns: 1149 entries, no. to COH.F.gamma.r.O1.s.O2
dtypes: float64(1144), int64(1), object(4)
memory usage: 8.3+ MB
Data info: None


In [8]:
summary = data.describe()
print(summary)

              no.         age   education          IQ  AB.A.delta.a.FP1  \
count  945.000000  945.000000  930.000000  932.000000        945.000000   
mean   473.000000   30.594804   13.438710  101.580472         20.182936   
std    272.942302   11.781592    2.550461   17.022414         11.282022   
min      1.000000   18.000000    0.000000   49.000000          3.272260   
25%    237.000000   21.730000   12.000000   91.000000         12.784872   
50%    473.000000   26.150000   13.000000  102.000000         17.065286   
75%    709.000000   35.450000   16.000000  114.000000         24.492760   
max    945.000000   71.880000   20.000000  145.000000         92.826192   

       AB.A.delta.b.FP2  AB.A.delta.c.F7  AB.A.delta.d.F3  AB.A.delta.e.Fz  \
count        945.000000       945.000000       945.000000       945.000000   
mean          21.177584        17.749553        18.901698        20.447079   
std           12.230662        10.003598         9.079482         9.742912   
min         

In [9]:
# load dataset into DataFrame
df = pd.read_csv('/Users/soukainagadir/Downloads/EEG.machinelearing_data_BRMH.csv')

# define function to rename columns
def reformat_name(name):
    '''
    reformat from XX.X.band.x.channel to band.channel or 
    COH.X.band.x.channel1.x.channel2 to COH.band.channel1.channel2
    '''
    splitted = name.split(sep='.')
    if len(splitted) < 5:
        return name
    if splitted[0] != 'COH':
        result = f'{splitted[2]}.{splitted[4]}'
    else:
        result = f'{splitted[0]}.{splitted[2]}.{splitted[4]}.{splitted[6]}'
    return result

# rename columns using the function
df.rename(reformat_name, axis=1, inplace=True)

# show the updated DataFrame
#print(df)
display(df)

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,delta.FP1,delta.FP2,...,COH.gamma.Pz.P4,COH.gamma.Pz.T6,COH.gamma.Pz.O1,COH.gamma.Pz.O2,COH.gamma.P4.T6,COH.gamma.P4.O1,COH.gamma.P4.O2,COH.gamma.T6.O1,COH.gamma.T6.O2,COH.gamma.O1.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,941,M,22.0,2014.8.28,13.0,116.0,Healthy control,Healthy control,41.851823,36.771496,...,82.905657,34.850706,63.970519,63.982003,51.244725,62.203684,62.062237,31.013031,31.183413,98.325230
941,942,M,26.0,2014.9.19,13.0,118.0,Healthy control,Healthy control,18.986856,19.401387,...,65.917918,66.700117,44.756285,49.787513,98.905995,54.021304,93.902401,52.740396,92.807331,56.320868
942,943,M,26.0,2014.9.27,16.0,113.0,Healthy control,Healthy control,28.781317,32.369230,...,61.040959,27.632209,45.552852,33.638817,46.690983,19.382928,41.050717,7.045821,41.962451,19.092111
943,944,M,24.0,2014.9.20,13.0,107.0,Healthy control,Healthy control,19.929100,25.196375,...,99.113664,48.328934,41.248470,28.192238,48.665743,42.007147,28.735945,27.176500,27.529522,20.028446


# Exploratory data analysis

1- Data includes:

patients' information
no. - patients unique id
sex - M (male) or F (female)
age
eeg.date - date of EEG
education - years of educations
IQ - intelligence quotient

2- diagnosis
main.disorder
specific.disorder

-QEEG PSD (power spectrum density) - absolute power value in each frequency band
-QEEG FC (functional connectivity) - coherence for each band and channel


In [10]:
patients_info = df.loc[:,'sex':'specific.disorder']
patients_info

,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder
0,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder
1,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder
2,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder
3,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder
4,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder
...,...,...,...,...,...,...,...
940,M,22.0,2014.8.28,13.0,116.0,Healthy control,Healthy control
941,M,26.0,2014.9.19,13.0,118.0,Healthy control,Healthy control
942,M,26.0,2014.9.27,16.0,113.0,Healthy control,Healthy control
943,M,24.0,2014.9.20,13.0,107.0,Healthy control,Healthy control


In [11]:
patients_info = df.loc[:,'sex':'specific.disorder']
patients_info

,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder
0,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder
1,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder
2,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder
3,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder
4,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder
...,...,...,...,...,...,...,...
940,M,22.0,2014.8.28,13.0,116.0,Healthy control,Healthy control
941,M,26.0,2014.9.19,13.0,118.0,Healthy control,Healthy control
942,M,26.0,2014.9.27,16.0,113.0,Healthy control,Healthy control
943,M,24.0,2014.9.20,13.0,107.0,Healthy control,Healthy control


In [12]:
delta_cols = [col for col in data.columns if 'delta' in col]
beta_cols = [col for col in data.columns if 'beta' in col]
theta_cols = [col for col in data.columns if 'theta' in col]
alpha_cols = [col for col in data.columns if 'alpha' in col]

print(f"Number of Delta Columns : {len(delta_cols)}")
print(f"Number of Beta Columns : {len(beta_cols)}")
print(f"Number of Theta Columns : {len(theta_cols)}")
print(f"Number of Alpha Columns : {len(alpha_cols)}")

Number of Delta Columns : 190
Number of Beta Columns : 380
Number of Theta Columns : 190
Number of Alpha Columns : 190


In [13]:
# checking for missing values
missing = df.isna().sum()
missing[missing > 0]

education        15
IQ               13
Unnamed: 122    945
dtype: int64

In [14]:
educ_na = df[df['education'].isna()]
iq_na = df[df['IQ'].isna()]
educ_iq_na = pd.concat([educ_na, iq_na]).drop_duplicates()
educ_iq_na

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,delta.FP1,delta.FP2,...,COH.gamma.Pz.P4,COH.gamma.Pz.T6,COH.gamma.Pz.O1,COH.gamma.Pz.O2,COH.gamma.P4.T6,COH.gamma.P4.O1,COH.gamma.P4.O2,COH.gamma.T6.O1,COH.gamma.T6.O2,COH.gamma.O1.O2
0,1,M,57.00,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
17,18,M,30.00,2013.9.27,NaN,86.0,Addictive disorder,Alcohol use disorder,12.443237,12.503703,...,85.080600,53.533875,62.817411,68.040950,71.834549,48.779049,80.516443,22.049743,76.277261,45.629506
21,22,M,20.00,2014.10.23,NaN,116.0,Addictive disorder,Alcohol use disorder,28.287190,22.412264,...,33.065475,24.351780,18.764173,32.610337,38.451805,20.933501,48.626712,16.258394,40.471735,24.175359
155,156,M,18.00,2013.12.7,NaN,103.0,Addictive disorder,Behavioral addiction disorder,16.548803,19.042179,...,93.825121,79.091595,83.132315,88.900783,87.508591,76.257681,89.811513,62.739950,84.729816,86.322844
173,174,M,23.00,2015.11.14,NaN,113.0,Healthy control,Healthy control,19.050158,23.277470,...,68.005720,54.685472,70.694110,62.676608,67.705314,46.667642,66.673782,29.689482,74.787580,51.039538
269,270,M,25.00,2015.9.23,NaN,85.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,6.186264,7.112847,...,72.743284,63.723838,61.257858,67.473157,77.664778,57.967084,73.898719,59.775191,81.941728,72.195171
270,271,M,34.00,2015.9.21,NaN,120.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,12.784872,15.922964,...,72.409132,49.265064,41.222401,68.010480,72.429188,26.188907,68.653874,18.072318,63.285321,41.822238
279,280,M,35.00,2016.6.2,NaN,NaN,Obsessive compulsive disorder,Obsessive compulsitve disorder,21.524573,22.227615,...,90.934877,93.568209,94.185299,92.271775,96.445879,91.470862,96.851513,92.296182,96.209585,96.660561
280,281,M,37.00,2016.6.27,NaN,110.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,11.971083,11.374465,...,91.150863,78.359575,64.861521,74.389105,89.863923,50.322649,75.469583,49.440177,84.022887,69.921675
281,282,M,22.00,2016.6.30,NaN,107.0,Obsessive compulsive disorder,Obsessive compulsitve disorder,12.516343,10.136242,...,96.419647,77.360976,95.156801,96.105427,87.746498,94.760227,97.579921,81.495805,86.770154,98.121870


In [15]:
drop_md = educ_iq_na['main.disorder'].value_counts().sort_index()
all_md = df['main.disorder'].value_counts().sort_index()[drop_md.index]
pd.concat([all_md, drop_md/all_md * 100], axis=1).set_axis(['all_data', 'na_percentage'], axis=1).sort_values('na_percentage', ascending=False)

,all_data,na_percentage
main.disorder,,
Obsessive compulsive disorder,46,13.043478
Addictive disorder,186,4.301075
Trauma and stress related disorder,128,3.906250
Healthy control,95,2.105263
Mood disorder,266,1.503759
Anxiety disorder,107,0.934579


In [16]:
display(df[['education', 'IQ']].agg(['mean', 'median']))
imputer = SimpleImputer(strategy='median')

,education,IQ
mean,13.43871,101.580472
median,13.00000,102.000000


In [17]:
#The percentage of data we will loose if we drop rows with missing data 
drop_md = educ_iq_na['main.disorder'].value_counts().sort_index()
all_md = df['main.disorder'].value_counts().sort_index()[drop_md.index]
pd.concat([all_md, drop_md/all_md * 100], axis=1).set_axis(['all_data', 'na_percentage'], axis=1).sort_values('na_percentage', ascending=False)

,all_data,na_percentage
main.disorder,,
Obsessive compulsive disorder,46,13.043478
Addictive disorder,186,4.301075
Trauma and stress related disorder,128,3.906250
Healthy control,95,2.105263
Mood disorder,266,1.503759
Anxiety disorder,107,0.934579


In [19]:
# We loose too much data (13%) on patients with obsessive compulsive disorder, which is one of the smallest 
#groups already. We should consider filling missing data. with median value 
educ_iq_medians = df[['education', 'IQ']].agg(np.nanmedian)
df[['education', 'IQ']] = df[['education', 'IQ']].fillna(educ_iq_medians)
missing = df.isna().sum()
missing[missing>0]

/var/folders/0x/p1p3991d1hvd020jfhdwpqkr0000gn/T/ipykernel_36753/535418922.py:3: FutureWarning: The provided callable <function nanmedian at 0x7ff6f827b310> is currently using DataFrame.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  educ_iq_medians = df[['education', 'IQ']].agg(np.nanmedian)


Unnamed: 122    945
dtype: int64

In [20]:
# checking the distribution of Target Variable
data['main.disorder'].value_counts()

main.disorder
Mood disorder                         266
Addictive disorder                    186
Trauma and stress related disorder    128
Schizophrenia                         117
Anxiety disorder                      107
Healthy control                        95
Obsessive compulsive disorder          46
Name: count, dtype: int64

In [21]:
#Let's rename some of the columns:

#XX.X.band.x.channel to band.channel
#COH.X.band.x.channel1.x.channel2 to COH.band.channel1.channel2

# function to rename
def reformat_name(name):
    '''
    reformat from XX.X.band.x.channel to band.channel or 
    COH.X.band.x.channel1.x.channel2 to COH.band.channel1.channel2
    '''
    splitted = name.split(sep='.')
    if len(splitted) < 5:
        return name
    if splitted[0] != 'COH':
        result = f'{splitted[2]}.{splitted[4]}'
    else:
        result = f'{splitted[0]}.{splitted[2]}.{splitted[4]}.{splitted[6]}'
    return result
# rename columns
data.rename(reformat_name, axis=1, inplace=True)
data

print(data)



     no. sex   age    eeg.date  education     IQ       main.disorder  \
0      1   M  57.0   2012.8.30        NaN    NaN  Addictive disorder   
1      2   M  37.0    2012.9.6        6.0  120.0  Addictive disorder   
2      3   M  32.0   2012.9.10       16.0  113.0  Addictive disorder   
3      4   M  35.0   2012.10.8       18.0  126.0  Addictive disorder   
4      5   M  36.0  2012.10.18       16.0  112.0  Addictive disorder   
..   ...  ..   ...         ...        ...    ...                 ...   
940  941   M  22.0   2014.8.28       13.0  116.0     Healthy control   
941  942   M  26.0   2014.9.19       13.0  118.0     Healthy control   
942  943   M  26.0   2014.9.27       16.0  113.0     Healthy control   
943  944   M  24.0   2014.9.20       13.0  107.0     Healthy control   
944  945   M  21.0  2015.10.23       13.0  105.0     Healthy control   

        specific.disorder  delta.FP1  delta.FP2  ...  COH.gamma.Pz.P4  \
0    Alcohol use disorder  35.998557  21.717375  ...        55

In [22]:
# Define the columns you want to select
selected_columns = ['no.', 'sex','age','education','specific.disorder','COH.beta.P3.O2', 'COH.alpha.C3.O2', 'gamma.C4', 'COH.beta.FP1.Fz' , 'delta.O1', 'COH.alpha.T4.P4', 'COH.delta.T5.Pz', 'COH.theta.T5.P3', 'COH.beta.F7.C3', 'COH.beta.FP1.C3', 'COH.beta.T4.T6', 'COH.delta.F7.T3','COH.beta.FP2.Fz','COH.theta.T4.P3','COH.alpha.T3.Cz',]


# Use loc method to select the rows and columns you want to print
important_features = data.loc[:, selected_columns]

# Print the selected data
print(important_features)

     no. sex   age  education     specific.disorder  COH.beta.P3.O2  \
0      1   M  57.0        NaN  Alcohol use disorder       28.713693   
1      2   M  37.0        6.0  Alcohol use disorder       35.236759   
2      3   M  32.0       16.0  Alcohol use disorder       28.984787   
3      4   M  35.0       18.0  Alcohol use disorder       28.703657   
4      5   M  36.0       16.0  Alcohol use disorder       51.399252   
..   ...  ..   ...        ...                   ...             ...   
940  941   M  22.0       13.0       Healthy control       61.117190   
941  942   M  26.0       13.0       Healthy control       33.275949   
942  943   M  26.0       16.0       Healthy control       10.459981   
943  944   M  24.0       13.0       Healthy control       18.051707   
944  945   M  21.0       13.0       Healthy control       34.332282   

     COH.alpha.C3.O2  gamma.C4  COH.beta.FP1.Fz   delta.O1  COH.alpha.T4.P4  \
0           8.921215  1.765493        69.707875  18.278916        25

In [23]:
# Create binary columns for specific disorder and sex
specific_dummies = pd.get_dummies(important_features['specific.disorder'], prefix='specific_disorder')
sex_dummies = pd.get_dummies(important_features['sex'], prefix='sex')

# Concatenate the binary columns to the original dataset
important_features = pd.concat([important_features, specific_dummies, sex_dummies], axis=1)

# Drop the original specific disorder and sex columns
important_features = important_features.drop(['specific.disorder','sex'], axis=1)


In [24]:
important_features.to_csv('/Users/soukainagadir/Downloads/updated_dataset.csv', index=False)

In [25]:
# Select the desired columns
selected_columns = ['no.','sex_F','sex_M', 'age','education','COH.beta.P3.O2', 'COH.alpha.C3.O2', 'gamma.C4', 'COH.beta.FP1.Fz' , 'delta.O1', 'COH.alpha.T4.P4', 'COH.delta.T5.Pz', 'COH.theta.T5.P3', 'COH.beta.F7.C3', 'COH.beta.FP1.C3', 'COH.beta.T4.T6', 'COH.delta.F7.T3','COH.beta.FP2.Fz','COH.theta.T4.P3','COH.alpha.T3.Cz','specific_disorder_Depressive disorder']
selected_data = important_features.loc[:, selected_columns] # using loc accessor

# Print the selected data
print(selected_data)

     no.  sex_F  sex_M   age  education  COH.beta.P3.O2  COH.alpha.C3.O2  \
0      1  False   True  57.0        NaN       28.713693         8.921215   
1      2  False   True  37.0        6.0       35.236759         6.500859   
2      3  False   True  32.0       16.0       28.984787        11.426603   
3      4  False   True  35.0       18.0       28.703657        18.566733   
4      5  False   True  36.0       16.0       51.399252        21.380211   
..   ...    ...    ...   ...        ...             ...              ...   
940  941  False   True  22.0       13.0       61.117190        10.249904   
941  942  False   True  26.0       13.0       33.275949        13.715571   
942  943  False   True  26.0       16.0       10.459981        52.583405   
943  944  False   True  24.0       13.0       18.051707        47.649958   
944  945  False   True  21.0       13.0       34.332282         3.563636   

     gamma.C4  COH.beta.FP1.Fz   delta.O1  ...  COH.delta.T5.Pz  \
0    1.765493       

In [26]:
selected_data.to_csv('/Users/soukainagadir/Downloads/Most-updated_dataset.csv', index=False)

1 --> Depressive disorder


0 --> Non-depressive disorder


Splitting the Features and Target

In [27]:
X = selected_data.drop(columns='specific_disorder_Depressive disorder', axis=1)
Y = selected_data['specific_disorder_Depressive disorder']

In [28]:
print(X)

     no.  sex_F  sex_M   age  education  COH.beta.P3.O2  COH.alpha.C3.O2  \
0      1  False   True  57.0        NaN       28.713693         8.921215   
1      2  False   True  37.0        6.0       35.236759         6.500859   
2      3  False   True  32.0       16.0       28.984787        11.426603   
3      4  False   True  35.0       18.0       28.703657        18.566733   
4      5  False   True  36.0       16.0       51.399252        21.380211   
..   ...    ...    ...   ...        ...             ...              ...   
940  941  False   True  22.0       13.0       61.117190        10.249904   
941  942  False   True  26.0       13.0       33.275949        13.715571   
942  943  False   True  26.0       16.0       10.459981        52.583405   
943  944  False   True  24.0       13.0       18.051707        47.649958   
944  945  False   True  21.0       13.0       34.332282         3.563636   

     gamma.C4  COH.beta.FP1.Fz   delta.O1  COH.alpha.T4.P4  COH.delta.T5.Pz  \
0    1.7

In [29]:
print(Y)

0      False
1      False
2      False
3      False
4      False
       ...  
940    False
941    False
942    False
943    False
944    False
Name: specific_disorder_Depressive disorder, Length: 945, dtype: bool


Splitting the Data into Training data & Test Data

#X_train= features of all the training data , 
#X_test= features of all the test data, 
#Y_train= the target of all traning data, 
#Y_test= the target of all the test data,
#test_size= how many percentage of the data you want as test data,
#stratify= so 1 and 0 will be distributed in an even manner,
#random_state=2: data willbe splitted in an organized way 

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [31]:
print(X.shape, X_train.shape, X_test.shape)

(945, 20) (756, 20) (189, 20)


In [32]:
# Create an imputer object
imputer = SimpleImputer(strategy='median')

# Fit the imputer to the training data
imputer.fit(X_train)

# Transform the training data by replacing missing values with the median
X_train = imputer.transform(X_train)

# Transform the test data using the same imputer object
X_test = imputer.transform(X_test)


Model Training

Logistic Regression

In [33]:
model = LogisticRegression()

In [34]:
# training the LogisticRegression model with Training data
model.fit(X_train, Y_train)

/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation

Accuracy Score

In [37]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [38]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.7883597883597884


In [39]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [40]:
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.783068783068783


Building a Predictive System

In [41]:
import numpy as np

# original input data
input_data = (1,57.0,28.713693,8.921215,1.765493,69.707875,18.278916,25.948707,5.452973,21.216558,32.219015,30.300284,23.934374,52.721991,59.120406,7.779092,25.160966)

# add zeros to make it have 20 elements
input_data_as_numpy_array = np.concatenate((np.zeros(3), np.asarray(input_data)))

# reshape the numpy array as we are predicting for only one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)

if (prediction[0]== 0):
  print('The Person does not have depression')
else:
  print('The Person has depression')


[False]
The Person does not have depression


In [42]:
import pandas as pd

# Load your dataset
data = pd.read_csv("/Users/soukainagadir/Downloads/Most-updated_dataset.csv")

# Remove the last column using iloc
data = data.iloc[:, :-1]

# Save the modified dataset (optional)
data.to_csv("/Users/soukainagadir/Downloads/important_features.csv", index=False)

# Display the modified dataset
print(data)


     no.  sex_F  sex_M   age  education  COH.beta.P3.O2  COH.alpha.C3.O2  \
0      1  False   True  57.0        NaN       28.713693         8.921215   
1      2  False   True  37.0        6.0       35.236759         6.500859   
2      3  False   True  32.0       16.0       28.984787        11.426603   
3      4  False   True  35.0       18.0       28.703657        18.566733   
4      5  False   True  36.0       16.0       51.399252        21.380211   
..   ...    ...    ...   ...        ...             ...              ...   
940  941  False   True  22.0       13.0       61.117190        10.249904   
941  942  False   True  26.0       13.0       33.275949        13.715571   
942  943  False   True  26.0       16.0       10.459981        52.583405   
943  944  False   True  24.0       13.0       18.051707        47.649958   
944  945  False   True  21.0       13.0       34.332282         3.563636   

     gamma.C4  COH.beta.FP1.Fz   delta.O1  COH.alpha.T4.P4  COH.delta.T5.Pz  \
0    1.7

Here I used cross-validation in this code. I specifically used k-fold cross-validation with k=5, which is the default value in both GridSearchCV and RandomizedSearchCV in scikit-learn

Cross-validation is a technique used in machine learning to evaluate the performance of a model on a limited sample size. It involves splitting the dataset into multiple folds, training the model on a subset of the folds and testing it on the remaining fold. This process is repeated for each fold, and the results are averaged to obtain a more robust estimate of the model's performance.

In [43]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score


# Perform data preprocessing and feature extraction steps
# ...


# Assuming you have already split, imputed, and scaled your data
# Assuming your features are in 'X' and target variable in 'y'
X = important_features.drop(columns=['specific_disorder_Depressive disorder'], axis=1)
y = important_features['specific_disorder_Depressive disorder']


# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Impute missing values
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the logistic regression model
model = LogisticRegression()

# Define the hyperparameters to fine-tune
param_grid = {
    'C': np.logspace(-4, 4, 20),
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Using GridSearchCV for extensive search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, Y_train)

print("Best parameters found by GridSearchCV:")
print(grid_search.best_params_)

# Using RandomizedSearchCV for random search
random_search = RandomizedSearchCV(model, param_grid, n_iter=50, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_scaled, Y_train)

print("Best parameters found by RandomizedSearchCV:")
print(random_search.best_params_)

# Evaluate the best model from GridSearchCV
best_model_grid = grid_search.best_estimator_
train_acc_grid = accuracy_score(best_model_grid.predict(X_train_scaled), Y_train)
test_acc_grid = accuracy_score(best_model_grid.predict(X_test_scaled), Y_test)

print("GridSearchCV - Accuracy on Training data:", train_acc_grid)
print("GridSearchCV - Accuracy on Test data:", test_acc_grid)

# Evaluate the best model from RandomizedSearchCV
best_model_random = random_search.best_estimator_
train_acc_random = accuracy_score(best_model_random.predict(X_train_scaled), Y_train)
test_acc_random = accuracy_score(best_model_random.predict(X_test_scaled), Y_test)

print("RandomizedSearchCV - Accuracy on Training data:", train_acc_random)
print("RandomizedSearchCV - Accuracy on Test data:", test_acc_random)


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require I

/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sou

Best parameters found by GridSearchCV:
{'C': 0.03359818286283781, 'penalty': 'l2', 'solver': 'liblinear'}
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found by RandomizedSearchCV:
{'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.615848211066026}
GridSearchCV - Accuracy on Training data: 1.0
GridSearchCV - Accuracy on Test data: 1.0
RandomizedSearchCV - Accuracy on Training data: 1.0
RandomizedSearchCV - Accuracy on Test data: 1.0


/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/soukainagadir/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sou

In [44]:
# Get the best estimator from GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions on the training dataset
train_preds = best_model.predict(X_train_scaled)

# Make predictions on the test dataset
test_preds = best_model.predict(X_test_scaled)

# Calculate the accuracy scores for training and test datasets
train_accuracy = accuracy_score(Y_train, train_preds)
test_accuracy = accuracy_score(Y_test, test_preds)

# Print the results
print(f"Accuracy on Training data: {train_accuracy}")
print(f"Accuracy on Test data: {test_accuracy}")


Accuracy on Training data: 1.0
Accuracy on Test data: 1.0


# Interface

In [45]:
!pip install ipywidgets


In [1]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np

# Define the function to predict depression
def predict_depression(input_data):
    input_data_as_numpy_array = np.asarray(input_data)
    input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
    input_data_std = scaler.transform(input_data_reshaped)

    prediction = best_model.predict(input_data_std)
    prediction_label = np.argmax(prediction)

    result = "The patient does have depression" if prediction_label == 1 else "The patient does not have depression"
    return result

# Define widgets
title = widgets.HTML(value="<h2>Depression Prediction</h2>", layout=widgets.Layout(margin="0 auto"))

input_data = widgets.Text(
    value="(0,0,0,1,57.0,28.713693,8.921215,1.765493,69.707875,18.278916,25.948707,5.452973,21.216558,32.219015,30.300284,23.934374,52.721991,59.120406,7.779092,25.160966,0,0,0,0,0,0,0,0,0,0,0)",
    placeholder="Enter input data here (comma-separated values):",
    description="Input Data:",
    layout=widgets.Layout(width="80%"),
)

run_button = widgets.Button(description="Predict Depression", button_style="success", layout=widgets.Layout(width="50%"))

output = widgets.Output()

# Define button event handler
def on_button_click(button):
    with output:
        output.clear_output()
        input_data_str = input_data.value.strip("()")
        input_data_tuple = tuple(map(float, input_data_str.split(",")))
        prediction = predict_depression(input_data_tuple)
        print(prediction)

# Bind the event handler to the button
run_button.on_click(on_button_click)

# Display the interface
widgets.VBox([title, input_data, widgets.HBox([run_button], layout=widgets.Layout(justify_content="center")), output])
